In [92]:
import minsearch
from openai import OpenAI
import json 
from elasticsearch import Elasticsearch

import os 
from groq import Groq 
from dotenv import load_dotenv 
load_dotenv()

client = Groq(api_key = os.environ.get('GROQ_API_KEY'))

In [93]:
es_client = Elasticsearch(
    ['http://localhost:9200'],
    headers={
        "Accept": "application/vnd.elasticsearch+json; compatible-with=8",
        "Content-Type": "application/json"
    }  
)

In [94]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

# # Check if index exists
# if not es.indices.exists(index=index_name):
#     es.indices.create(index=index_name)
# else:
#     print(f"Index '{index_name}' already exists.")

# Delete if exists
if es_client.indices.exists(index=index_name, body=index_settings):
    es_client.indices.delete(index=index_name, body=index_settings)

# Recreate the index

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [95]:

file_path = '/Users/pawanchahar/LLM_ZoomCamp/01-intro/documents.json'
with open(file_path , 'rt') as f_in:
    docs_raw = json.load(f_in)
    

In [96]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)


In [97]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [98]:
def search(query):
    boost = {'question': 3.0 , 'section' : 0.5}

    results = index.search(
        query = query,
        boost_dict = boost,
        num_results = 5
    )

    return results 

In [99]:

def build_prompt(query, results):
    prompt_template = """ 
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.


QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""

    for doc in results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    
    return prompt 

In [100]:
def llm(prompt):
    response = client.chat.completions.create(
        model= 'llama3-70b-8192',
        messages= [{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [101]:
query = 'the course already started. Can I still enroll?'


def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [102]:
query = 'the course already started. Can I still enroll?'

In [103]:
from tqdm.auto import tqdm

In [104]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [105]:
def elastic_search(query):

    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index= index_name, body= search_query)

    result_doc = []

    for hit in response['hits']['hits']:
        result_doc.append(hit['_source'])

    return result_doc

In [106]:

def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [107]:
query = 'I just disovered the course. Can I still join it?'

rag(query)

'According to the FAQ database, the answer to your question "I just discovered the course. Can I still join it?" is:\n\nYes, you can still join the course. You don\'t need a confirmation email, and you can start submitting homework without registering. Registration is only to gauge interest before the start date.'